# Topic Modelling

## Packages

In [1]:
# Import Libraries
import pandas as pd
import numpy as np

from gensim.corpora import Dictionary
from gensim.models import LdaModel
import pyLDAvis.gensim_models

/Users/cindy/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


## Load data

In [3]:
# Load data
data_folder = '../data/'
transcript_df = pd.read_csv(data_folder + 'processed_transcripts.csv')

## Bag of Words (BoW)

In [9]:
# Create a gensim dictionary
dictionary = Dictionary(transcript_df['lemmatized'])

In [10]:
# Create a bag-of-words representation of the documents
corpus = [dictionary.doc2bow(text) for text in transcript_df['lemmatized']]

## Topic Modelling: LDA

In [11]:
# Train an LDA model on the corpus
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10)

In [12]:
# Visualize the results using pyLDAvis
pyLDAvis.enable_notebook()
vis_data = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.save_html(vis_data, '../results/topic_modeling.html')

/Users/cindy/opt/anaconda3/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


---